In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ECE 180D/output_truth.csv')    # reading the csv file
train.head()      # printing first five rows of the file

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ECE 180D/output_truth.csv'

In [ ]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('/content/drive/MyDrive/ECE 180D/output_mel_spectrograms/'+train['File'][i],target_size=(465,308,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 3900/3900 [40:45<00:00,  1.59it/s]


In [ ]:
X.shape

In [ ]:
plt.imshow(X[2])

In [ ]:
y = np.array(train.drop(['File'],axis=1))
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(465,308,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=64)

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/content/drive/MyDrive/ECE 180D/model.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions: ", predictions)
print("actual labels: ", y_test)


In [ ]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/ECE 180D/model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare your input data (this will depend on your model's input requirements)
input_data = X_test[0:10] # Prepare input data for inference

# Define the number of iterations for your loop
num_iterations = 10

# Run inference in a loop
for i in range(num_iterations):
    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data[i])

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Process output data (this will depend on your model's output)
    # For example, you can print the output
    print("Output:", output_data)
    print("label:", y_test[i])